In [41]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [42]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('data.csv')

In [ ]:
print(df.head())

In [44]:
# Drop the 'Remark' column as it is empty
df = df.drop(columns=['Remark'])

In [45]:
# Convert 'Buy date' and 'Closing date' to datetime
df['Buy date'] = pd.to_datetime(df['Buy date'], format='%d-%m-%Y')
df['Closing date'] = pd.to_datetime(df['Closing date'], format='%d-%m-%Y')

In [ ]:
# Sort data by 'Buy date' to ensure cumulative calculations are in order
df = df.sort_values(by='Buy date')

# Group by 'Buy date' and sum 'Buy value' and 'Unrealised P&L' for each date
investment_over_time = df.groupby('Buy date')[['Buy value', 'Unrealised P&L']].sum().reset_index()

# Calculate cumulative amount invested and cumulative P&L over time
investment_over_time['Cumulative Invested'] = investment_over_time['Buy value'].cumsum()
investment_over_time['Cumulative P&L'] = investment_over_time['Unrealised P&L'].cumsum()

# Create the line chart to show amount invested vs total P&L over time
fig = px.line(investment_over_time, x='Buy date', 
              y=['Cumulative Invested', 'Cumulative P&L'],
              labels={'value': 'Amount', 'variable': 'Type', 'Buy date': 'Date'},
              title='Amount Invested vs Total P&L Over Time')

# Show the plot
fig.show()

In [ ]:
# Aggregated Stock Allocation by Value
stock_value_allocation = df.groupby('Stock name')[['Buy value', 'Closing value']].sum().reset_index()

fig2 = px.pie(stock_value_allocation, names='Stock name', values='Closing value',
              title='Stock Allocation by Value')

# Use `hole` to create a donut-like pie chart
fig2.update_traces(hole=.4, hoverinfo="label+percent+name")

fig2.show()

In [ ]:
# Unrealized P&L (Aggregated by Stock)
stock_pnl = df.groupby('Stock name')['Unrealised P&L'].sum().reset_index()
# Add a column for color based on Unrealised P&L
stock_pnl['Color'] = stock_pnl['Unrealised P&L'].apply(lambda x: 'Profit' if x >= 0 else 'Loss')
stock_pnl_sorted = stock_pnl.sort_values(by='Unrealised P&L', ascending=False)

fig3 = px.bar(stock_pnl_sorted, x='Stock name', y='Unrealised P&L',
              color='Color',
              color_discrete_map={'Profit': 'green', 'Loss': 'red'},
              title='Unrealized P&L')
fig3.show()

In [ ]:
# Total Quantity Held Per Stock
total_quantity = df.groupby('Stock name')['Quantity'].sum().reset_index()
total_quantity = total_quantity.sort_values(by='Quantity', ascending=False)

fig4 = px.bar(total_quantity, x='Stock name', y='Quantity',
              title='Total Quantity Held Per Stock')
fig4.show()

In [ ]:
# Group by 'Stock name' and sum 'Buy value' and 'Unrealised P&L' for each stock
value_comparison = df.groupby('Stock name')[['Buy value', 'Unrealised P&L']].sum().reset_index()

# Separate positive and negative Unrealised P&L values for conditional coloring
positive_pnl = value_comparison['Unrealised P&L'].apply(lambda x: max(x, 0))
negative_pnl = value_comparison['Unrealised P&L'].apply(lambda x: min(x, 0))

# Create the stacked bar chart
fig5 = go.Figure(data=[
    go.Bar(name='Buy value', x=value_comparison['Stock name'], y=value_comparison['Buy value']),
    go.Bar(name='Unrealised P&L (negative)', x=value_comparison['Stock name'], y=negative_pnl),
    go.Bar(name='Unrealised P&L (positive)', x=value_comparison['Stock name'], y=positive_pnl)
])

# Update layout to make the bars stacked
fig5.update_layout(barmode='stack', title='Aggregated Buy vs Unrealised P&L Comparison (Per Stock)')

# Show the plot
fig5.show()

In [ ]:
# Performance of Stocks by Time Held (Across Trades)
df['Holding Period (days)'] = (df['Closing date'] - df['Buy date']).dt.days

fig6 = px.scatter(df, x='Holding Period (days)', y='Unrealised P&L', color='Stock name',
                  title='Performance of Stocks by Time Held')
fig6.show()

In [ ]:
# P&L Percentage by Stock (Aggregated)
pnl_percentage = df.groupby('Stock name')[['Unrealised P&L', 'Buy value']].sum().reset_index()
pnl_percentage['P&L Percentage'] = (pnl_percentage['Unrealised P&L'] / pnl_percentage['Buy value']) * 100

fig7 = px.bar(pnl_percentage, x='Stock name', y='P&L Percentage',
              title='P&L Percentage by Stock')
fig7.show()